In [2]:
import math
import numpy as np
import pandas as pd

In [3]:
fdata = pd.read_csv('data_histories.csv', index_col="dp_folio")
fdata.head()

,id_sexo,Aedad,AAedad,Apuesto,id_gestud,AIMC,fuma,fuma_act,ejer_act,ejer1,...,locout5,locout10,locout20,locout30,rest_act,rest1,rest5,rest10,rest20,rest30
dp_folio,,,,,,,,,,,,,,,,,,,,,
1,F,51,6,Admin,CarTec,4,1,3,0,2,...,1,1,1,1,0,0,0,0,0,0
2,F,38,4,Sec,Bach,3,2,-1,0,0,...,0,0,0,0,1,3,0,0,0,0
3,F,34,3,Int,Sec,5,1,1,0,0,...,1,-1,-1,-1,0,0,0,-1,-1,-1
4,M,63,8,Jef,CarTec,4,2,-1,2,2,...,0,0,0,-1,0,0,0,0,0,0
5,M,42,4,EM,Sec,3,1,2,2,2,...,0,0,0,0,0,0,0,0,0,0


### Functions for clusterization

In [4]:
# Function to clusterize categories of a certain feature, and add the new clusterized feature as a new column
# Clusters should be an input of the form {cluster_A: {categories}, cluster_B: [categories]}
# Ex. obesity = {0:[1,2,3], 1:[4,5,6]}
def clusterizeDiscrete(feature, clusters, new_name, data):
    new_data = data.copy()
    original_list = data.loc[1:1080, feature]
    new_list = []
    for index in original_list.index:
        if original_list[index] == -1 or original_list[index] == "-1":
            new_list.append("N")
            continue
        for cluster in clusters:
            if original_list[index] in clusters[cluster]:
                new_list.append(cluster)
    new_data[new_name] = new_list
    return new_data

# Function to clusterize categories of a certain continous feature, and add the new clusterized feature as a
# new column
# Clusters shoud be an input of the form {cluster_A: {lambdaFunction1}, cluster_B: lambdaFunction2}
def clusterizeContinuous(feature, clusters, new_name, data):
    new_data = data.copy()
    original_list = data.loc[1:1080, feature]
    new_list = []
    for index in original_list.index:
        if original_list[index] == -1 or original_list[index] == "-1":
            new_list.append("N")
            continue
        for cluster in clusters:
            if eval(clusters[cluster])(original_list[index]):
                new_list.append(cluster)
                break
    new_data[new_name] = new_list
    return new_data
        

# Set of auxiliary high-order functions that will evaluate the conditions to binarize a history
def lessThan(num):
    return lambda n: n < num

def lessQThan(num):
    return lambda n: n <= num

def greaterThan(num):
    return lambda n: n > num

def greaterQThan(num):
    return lambda n: n >= num

def between(num1, num2):
    return lambda n: n <= num2 and n >= num1

### Functions for counting/calculating probabilities

In [5]:
# Functions to count ocurrences for a category of a feature (NX)

# Count the number of instances inside the database whose feature X = category
# Ex: getNX('AIMC', 3, fdata)
def getNX(feature, category, data):
    count = 0
    for index in data.index:
        if data.loc[index][feature] == category:
            count = count + 1
    return count

# Count the number of instances inside the database whose feature X_1 = category_1 and feature X_2 = category_2
# Ex: getNCX('AIMC', 3, 'Obesidad', 1, fdata)
def getNCX(feature_1, category_1, feature_2, category_2, data):
    count = 0
    for index in data.index:
        if data.loc[index][feature_1] == category_1 and data.loc[index][feature_2] == category_2:
            count = count + 1
    return count

# Get a conditional probability P(F_1 = C_1 | F_2 = C_2)
def getCondProb(feature_1, category_1, feature_2, category_2, data):
    ncx = getNCX(feature_1, category_1, feature_2, category_2, data)
    nx = getNX(feature_2, category_2, data)
    if nx != 0:
        p = ncx / float(nx)
    else:
        p = 0
    #return {'P': p, 'nx': nx, 'ncx': ncx}
    return p

def getEpsilon(feature, category, classFeature, classCategory, data):
    n = len(data)
    nx = getNX(feature, category, data)
    nc = getNX(classFeature, classCategory, data)
    ncx = getNCX(feature, category, classFeature, classCategory, data)
    if n != 0 and nx != 0:
        pc = nc / float(n)
        pcx = ncx / float(nx)
        epsilon = nx * (pcx - pc) / math.sqrt(nx * pc * (1 - pc))
    else:
        epsilon = 0
    #print 'Epsilon :' + str(epsilon)
    return {'epsilon': epsilon, 'nx': nx, 'ncx': ncx, 'nc': nc}

In [6]:
# PENDING: Modularize this function, as it only works now with binary type variables
def transitionProbabilities(feature, data):
    years = ["30", "20", "10", "5", "1", "0"]
    transitionProbs = {}
    transitionProbs["labels"] = []
    transitionProbs["probs"] = []
    transitionProbs["pattern"] = []
    for i in range(len(years)):
        for j in range(i):
            feature_1 = feature + years[i] + "B"
            feature_2 = feature + years[j] + "B"
            f_1 = "a_" + years[i]
            f_2 = "a_" + years[j]
            label_1 = f_1 + ": A | " + f_2 + ": A "
            label_2 = f_1 + ": B | " + f_2 + ": A "
            label_3 = f_1 + ": A | " + f_2 + ": B "
            label_4 = f_1 + ": B | " + f_2 + ": B "
            prob_1 = getCondProb(feature_1, "A", feature_2, "A", data)
            prob_2 = getCondProb(feature_1, "B", feature_2, "A", data)
            prob_3 = getCondProb(feature_1, "A", feature_2, "B", data)
            prob_4 = getCondProb(feature_1, "B", feature_2, "B", data)
            transitionProbs["labels"].append(label_1)
            transitionProbs["labels"].append(label_2)
            transitionProbs["labels"].append(label_3)
            transitionProbs["labels"].append(label_4)
            transitionProbs["probs"].append(prob_1)
            transitionProbs["probs"].append(prob_2)
            transitionProbs["probs"].append(prob_3)
            transitionProbs["probs"].append(prob_4)
            transitionProbs["pattern"].append("A->A")
            transitionProbs["pattern"].append("A->B")
            transitionProbs["pattern"].append("B->A")
            transitionProbs["pattern"].append("B->B")
            #print label_1 + str(prob_1)
            #print label_2 + str(prob_2)
            #print label_3 + str(prob_3)
            #print label_4 + str(prob_4)
    return pd.DataFrame(data = transitionProbs)

# Function to explore subgroups? 

### Clusters

In [7]:
# Clusterize between obese and non obese
obesity = {0:[1,2,3], 1:[4,5,6]}
ndata = clusterizeDiscrete("AIMC", obesity, "Obesity", fdata)

# Clusterize between degrees of study (higher degree vs. non higher degree)
degree = {0:["Prim", "Sec", "Bach", "CarTec", "Otro"], 1:["Lic", "Mast", "Doc", "PDoc"]}
ndata = clusterizeDiscrete("id_gestud", degree, "hdegree", ndata)

# Clusterize excercise features
ejer = {"A":"greaterQThan(2.5)", "B":"lessThan(2.5)"}
ndata = clusterizeContinuous("ejer_act", ejer, "ejer0B", ndata)
ndata = clusterizeContinuous("ejer1", ejer, "ejer1B", ndata)
ndata = clusterizeContinuous("ejer5", ejer, "ejer5B", ndata)
ndata = clusterizeContinuous("ejer10", ejer, "ejer10B", ndata)
ndata = clusterizeContinuous("ejer20", ejer, "ejer20B", ndata)
ndata = clusterizeContinuous("ejer30", ejer, "ejer30B", ndata)

# Clusterize stress features
estres = {"A":"greaterQThan(4)", "B":"lessThan(4)"}
ndata = clusterizeContinuous("estres_act", estres, "estres0B", ndata)
ndata = clusterizeContinuous("estres1", estres, "estres1B", ndata)
ndata = clusterizeContinuous("estres5", estres, "estres5B", ndata)
ndata = clusterizeContinuous("estres10", estres, "estres10B", ndata)
ndata = clusterizeContinuous("estres20", estres, "estres20B", ndata)
ndata = clusterizeContinuous("estres30", estres, "estres30B", ndata)

# Clusterize weight features
peso = {"A":"greaterQThan(4)", "B":"lessThan(4)"}
ndata = clusterizeContinuous("peso_act", peso, "peso0B", ndata)
ndata = clusterizeContinuous("peso1", peso, "peso1B", ndata)
ndata = clusterizeContinuous("peso5", peso, "peso5B", ndata)
ndata = clusterizeContinuous("peso10", peso, "peso10B", ndata)
ndata = clusterizeContinuous("peso20", peso, "peso20B", ndata)
ndata = clusterizeContinuous("peso30", peso, "peso30B", ndata)

# Clusterize weight features
condi = {"A":"greaterQThan(4)", "B":"lessThan(4)"}
ndata = clusterizeContinuous("condi_act", condi, "condi0B", ndata)
ndata = clusterizeContinuous("condi1", condi, "condi1B", ndata)
ndata = clusterizeContinuous("condi5", condi, "condi5B", ndata)
ndata = clusterizeContinuous("condi10", condi, "condi10B", ndata)
ndata = clusterizeContinuous("condi20", condi, "condi20B", ndata)
ndata = clusterizeContinuous("condi30", condi, "condi30B", ndata)

In [7]:
ndata.head()

,id_sexo,Aedad,AAedad,Apuesto,id_gestud,AIMC,fuma,fuma_act,ejer_act,ejer1,...,peso5B,peso10B,peso20B,peso30B,condi0B,condi1B,condi5B,condi10B,condi20B,condi30B
dp_folio,,,,,,,,,,,,,,,,,,,,,
1,F,51,6,Admin,CarTec,4,1,3,0,2,...,B,B,B,A,B,A,A,A,A,A
2,F,38,4,Sec,Bach,3,2,-1,0,0,...,A,B,B,B,A,A,A,A,A,A
3,F,34,3,Int,Sec,5,1,1,0,0,...,A,A,B,A,B,B,B,B,A,A
4,M,63,8,Jef,CarTec,4,2,-1,2,2,...,B,B,B,B,B,B,A,A,A,A
5,M,42,4,EM,Sec,3,1,2,2,2,...,B,B,A,A,B,B,A,A,A,A


### Changes in excercise patterns

In [120]:
# All population
tp_allex = transitionProbabilities("ejer", ndata)
tp_allex.to_csv(path_or_buf="ejer_all.csv", index=False)

In [163]:
# Non Obese vs. Obese
# Get probabilities from both groups into a single DF
tp_obex = transitionProbabilities("ejer", ndata[ndata.Obesity == 0])
tp_obexc = transitionProbabilities("ejer", ndata[ndata.Obesity == 1])
tp_obex["probsc"] = tp_obexc.loc[:, "probs"]
tp_obex = tp_obex.rename(columns={"probs":"nobese", "probsc":"obese"})
tp_obex.to_csv(path_or_buf="ejer_obese_vs_nobese.csv", index=False)

In [172]:
# Non Higher Degree vs. Higher degree 
tp_degex = transitionProbabilities("ejer", ndata[ndata.hdegree == 0])
tp_degexc = transitionProbabilities("ejer", ndata[ndata.hdegree == 1])
tp_degex["probsc"] = tp_degexc.loc[:, "probs"]
tp_degex = tp_degex.rename(columns={"probs":"nhdegree", "probsc":"hdegree"})
tp_degex.to_csv(path_or_buf="ejer_nhdegree_vs_hdegree.csv", index=False)

In [211]:
tp_degex[(tp_degex.pattern == 'A->A')]

,labels,pattern,nhdegree,hdegree
0,a_20: A | a_30: A,A->A,0.883495,0.869159
4,a_10: A | a_30: A,A->A,0.718447,0.761682
8,a_10: A | a_20: A,A->A,0.786164,0.838710
12,a_5: A | a_30: A,A->A,0.592233,0.700935
16,a_5: A | a_20: A,A->A,0.578616,0.700880
20,a_5: A | a_10: A,A->A,0.727749,0.776119
24,a_1: A | a_30: A,A->A,0.368932,0.570093
28,a_1: A | a_20: A,A->A,0.396226,0.563050
32,a_1: A | a_10: A,A->A,0.445026,0.587065
36,a_1: A | a_5: A,A->A,0.541436,0.671875


In [212]:
tp_all[(tp_all.pattern == 'A->A')]

,labels,pattern,probs
0,a_20: A | a_30: A,A->A,0.873817
4,a_10: A | a_30: A,A->A,0.747634
8,a_10: A | a_20: A,A->A,0.822000
12,a_5: A | a_30: A,A->A,0.665615
16,a_5: A | a_20: A,A->A,0.662000
20,a_5: A | a_10: A,A->A,0.760540
24,a_1: A | a_30: A,A->A,0.504732
28,a_1: A | a_20: A,A->A,0.510000
32,a_1: A | a_10: A,A->A,0.541315
36,a_1: A | a_5: A,A->A,0.630088


### Changes in stress patterns

In [137]:
# All population
tp_allst = transitionProbabilities("estres", ndata)
tp_allst.to_csv(path_or_buf="estres_all.csv", index=False)

In [146]:
# Non Obese vs. Obese
# Get probabilities from both groups into a single DF
tp_obst = transitionProbabilities("estres", ndata[ndata.Obesity == 0])
tp_obstc = transitionProbabilities("estres", ndata[ndata.Obesity == 1])
tp_obst["probsc"] = tp_obstc.loc[:, "probs"]
tp_obst = tp_obst.rename(columns={"probs":"nobese", "probsc":"obese"})
tp_obst.to_csv(path_or_buf="estres_obese_vs_nobese.csv", index=False)

In [175]:
# Non Higher degree vs. Higher degree
tp_degst = transitionProbabilities("estres", ndata[ndata.hdegree == 0])
tp_degstc = transitionProbabilities("estres", ndata[ndata.hdegree == 1])
tp_degst["probsc"] = tp_degstc.loc[:, "probs"]
tp_degst = tp_degst.rename(columns={"probs":"nhdegree", "probsc":"hdegree"})
tp_degst.to_csv(path_or_buf="estres_hdegree_vs_nhdegree.csv", index=False)

In [210]:
tp_degst[(tp_degst.pattern == 'A->B')]

,labels,pattern,nhdegree,hdegree
1,a_20: B | a_30: A,A->B,0.497041,0.608833
5,a_10: B | a_30: A,A->B,0.810651,0.810726
9,a_10: B | a_20: A,A->B,0.605769,0.657895
13,a_5: B | a_30: A,A->B,0.757396,0.611987
17,a_5: B | a_20: A,A->B,0.576923,0.532895
21,a_5: B | a_10: A,A->B,0.292683,0.255474
25,a_1: B | a_30: A,A->B,0.585799,0.372240
29,a_1: B | a_20: A,A->B,0.557692,0.361842
33,a_1: B | a_10: A,A->B,0.463415,0.284672
37,a_1: B | a_5: A,A->B,0.292035,0.209486


### Changes in weight patterns

In [145]:
# All population
tp_allwg = transitionProbabilities("peso", ndata)
tp_allwg.to_csv(path_or_buf="peso_all.csv", index=False)

In [147]:
# Non Obese vs. Obese
# Get probabilities from both groups into a single DF
tp_obwg = transitionProbabilities("peso", ndata[ndata.Obesity == 0])
tp_obwgc = transitionProbabilities("peso", ndata[ndata.Obesity == 1])
tp_obwg["probsc"] = tp_obwgc.loc[:, "probs"]
tp_obwg = tp_obwg.rename(columns={"probs":"nobese", "probsc":"obese"})
tp_obwg.to_csv(path_or_buf="peso_obese_vs_nobese.csv", index=False)

In [178]:
# Non Higher degree vs. Higher degree
tp_degwg = transitionProbabilities("peso", ndata[ndata.hdegree == 0])
tp_degwgc = transitionProbabilities("peso", ndata[ndata.hdegree == 1])
tp_degwg["probsc"] = tp_degwgc.loc[:, "probs"]
tp_degwg = tp_degwg.rename(columns={"probs":"nhdegree", "probsc":"hdegree"})
tp_degwg.to_csv(path_or_buf="peso_hdegree_vs_nhdegree.csv", index=False)

In [199]:
tp_degwg[(tp_degwg.pattern == 'A->B')]

,labels,pattern,nhdegree,hdegree
1,a_20: B | a_30: A,A->B,0.476821,0.597973
5,a_10: B | a_30: A,A->B,0.708609,0.814189
9,a_10: B | a_20: A,A->B,0.500000,0.601399
13,a_5: B | a_30: A,A->B,0.615894,0.722973
17,a_5: B | a_20: A,A->B,0.500000,0.608392
21,a_5: B | a_10: A,A->B,0.177570,0.215385
25,a_1: B | a_30: A,A->B,0.450331,0.604730
29,a_1: B | a_20: A,A->B,0.421569,0.566434
33,a_1: B | a_10: A,A->B,0.214953,0.307692
37,a_1: B | a_5: A,A->B,0.151685,0.208531


### Changes in physical condition patterns

In [160]:
# All population
tp_allcn = transitionProbabilities("condi", ndata)
tp_allcn.to_csv(path_or_buf="cond_all.csv", index=False)

In [161]:
# Non Obese vs. Obese
# Get probabilities from both groups into a single DF
tp_obcn = transitionProbabilities("condi", ndata[ndata.Obesity == 0])
tp_obcnc = transitionProbabilities("condi", ndata[ndata.Obesity == 1])
tp_obcn["probsc"] = tp_obcnc.loc[:, "probs"]
tp_obcn = tp_obcn.rename(columns={"probs":"nobese", "probsc":"obese"})
tp_obcn.to_csv(path_or_buf="condi_obese_vs_nobese.csv", index=False)

In [180]:
# Non Higher degree vs. Higher degree
tp_degcn = transitionProbabilities("condi", ndata[ndata.hdegree == 0])
tp_degcnc = transitionProbabilities("condi", ndata[ndata.hdegree == 1])
tp_degcn["probsc"] = tp_degcnc.loc[:, "probs"]
tp_degcn = tp_degcn.rename(columns={"probs":"nhdegree", "probsc":"hdegree"})
tp_degcn.to_csv(path_or_buf="condi_hdegree_vs_nhdegree.csv", index=False)

In [184]:
tp_degcn[(tp_degcn.pattern == 'A->A')]

,labels,pattern,nhdegree,hdegree
0,a_20: A | a_30: A,A->A,0.958128,0.900498
4,a_10: A | a_30: A,A->A,0.812808,0.802653
8,a_10: A | a_20: A,A->A,0.841432,0.859232
12,a_5: A | a_30: A,A->A,0.633005,0.648425
16,a_5: A | a_20: A,A->A,0.654731,0.698355
20,a_5: A | a_10: A,A->A,0.756757,0.768145
24,a_1: A | a_30: A,A->A,0.337438,0.504146
28,a_1: A | a_20: A,A->A,0.347826,0.517367
32,a_1: A | a_10: A,A->A,0.387387,0.546371
36,a_1: A | a_5: A,A->A,0.488550,0.659259


#### Further Queries

In [208]:
getEpsilon("peso0B", "B", "Obesity", 1, ndata)

{'epsilon': -10.4412041435158, 'nc': 228, 'ncx': 8, 'nx': 478}

In [14]:
ndata[(ndata.peso0B == "A") & (ndata.Obesity == 1)].to_csv(path_or_buf="test.csv")

Definimos la variable $\xi_1 = OD$, es decir, como la variable compuesta de obesidad y grado. Fijamos D = 0 para estudiar si los patrones vistos anteriormente respecto a la preservación/cambio de hábitos, se mantiene con las personas obesas/no obesas que no tienen un grado de estudios superior.

In [18]:
tp_xi = transitionProbabilities("condi", ndata[(ndata.Obesity == 0) & (ndata.hdegree == 0)])
tp_xic = transitionProbabilities("condi", ndata[(ndata.Obesity == 1) & (ndata.hdegree == 0)])
tp_xi["probsc"] = tp_xic.loc[:, "probs"]
tp_xi = tp_xi.rename(columns={"probs":"nobese", "probsc":"obese"})

In [20]:
tp_xi.to_csv(path_or_buf="ejer_obese_vs_nobese_nhdegree.csv", index=False)

In [36]:
getEpsilon("hdegree", 1, "Obesity", 1, ndata)

{'epsilon': -4.087350662449773, 'nc': 228, 'ncx': 93, 'nx': 638}